# Get Traffic Flow Tile

For a given latitude, longitude at a particular zoom level, it returns the traffic flow from the map tile.
- https://docs.microsoft.com/en-us/rest/api/maps/traffic/gettrafficflowtile
- https://docs.microsoft.com/en-us/rest/api/maps/render/getmaptile
- https://docs.microsoft.com/en-us/azure/azure-maps/zoom-levels-and-tile-grid?tabs=csharp


- https://macwright.org/2015/03/23/geojson-second-bite.html
- https://github.com/jazzband/geojson
- https://github.com/micolous/geojsontools
- https://medium.com/@sumit.arora/what-is-geojson-geojson-basics-visualize-geojson-open-geojson-using-qgis-open-geojson-3432039e336d

requirements:
- https://github.com/tilezen/mapbox-vector-tile. 


In [1]:
# importing the requests library
import math
import requests
import numpy as np
import mapbox_vector_tile

In [2]:
def tileCalculater(latitude, longitude ,tileSize, zoom):
    
    print("1. The latitude, longitude: {0}, {1}".format(latitude,longitude))
    sinLatitude = np.sin(latitude * np.pi/180)
    
    pixelX = ((longitude + 180) / 360) * tileSize * np.power(2, zoom)
    pixelY = (0.5 - np.log((1 + sinLatitude) / (1 - sinLatitude)) / (4 * np.pi)) * tileSize * np.power(2, zoom)
    
    mapWidth = tileSize * np.power(2, zoom)
    mapHeight = mapWidth

    numberOfTilesWide = np.power(2, zoom)
    numberOfTilesHigh = numberOfTilesWide
    
    print("2. The mapWidth and numberOfTiles: {0}, {1}".format(mapWidth,numberOfTilesHigh))

    tileX = math.floor(pixelX / tileSize)
    tileY = math.floor(pixelY / tileSize)
    print("3. The tileX and tileY estimated.")
    
    return tileX, tileY

In [3]:
def trafficFlow(tileX, tileY):

    """
    API endpoint 
    """

    tileFormat = "pbf"
    style = "relative"
    key = "yf8upXHhjg4n0O5hf-i24-ZKPZN5kRbE4gGT-_3_TOU"

    URL = "https://atlas.microsoft.com/traffic/flow/tile/{0}?api-version=1.0&style={1}&tileSize={2}&zoom={3}&subscription-key={4}&x={5}&y={6}".format(tileFormat,
                                                                                                                                                  style,
                                                                                                                                                  tileSize,
                                                                                                                                                  zoom,
                                                                                                                                                  key,
                                                                                                                                                  tileX,
                                                                                                                                                  tileY) 
    out = requests.get(url=URL)
    tileLayer = mapbox_vector_tile.decode(out.content)
    trfficData = tileLayer["Traffic flow"]
    
    return trfficData

## Ricky Brundritt from Microsoft about Zoom Level:

I personally wouldn’t use a zoom level any closer than 15 as no additional traffic data will be brought in at lower zoom levels. In terms of calculating x/y tile grids at a zoom level when you have a coordinate, you first convert the coordinate into a global pixel. A global pixel is the Mercator pixel coordinate on the map for the specified zoom level when looking at the whole globe. From there you can convert the global pixel into x/y tile grid values by dividing the pixel values by 256 and rounding off.

In [4]:
if __name__ == "__main__": 
    
    zoom = 15
    tileSize = 512

    latitude, longitude = 52.519819, 13.297018 # a random coordinate in Berlin
    
    tileX, tileY = tileCalculater(latitude, longitude ,tileSize, zoom)
    trfficData = trafficFlow(tileX, tileY)

1. The latitude, longitude: 52.519819, 13.297018
2. The mapWidth and numberOfTiles: 16777216, 32768
3. The tileX and tileY estimated.


In [5]:
trfficData

{'extent': 4096,
 'version': 2,
 'features': [{'geometry': {'type': 'LineString',
    'coordinates': [[2372, 3178],
     [2340, 3152],
     [2328, 3140],
     [2246, 2916],
     [2218, 2816],
     [2204, 2754],
     [2194, 2610],
     [2194, 2528],
     [2202, 2410],
     [2224, 2228],
     [2260, 2004]]},
   'properties': {'road_type': 'Secondary road',
    'traffic_level': 0.3880000114440918,
    'traffic_road_coverage': 'full'},
   'id': 0,
   'type': 2},
  {'geometry': {'type': 'LineString',
    'coordinates': [[2234, 2128],
     [2240, 2126],
     [2124, 2124],
     [2036, 2112],
     [1016, 2012],
     [840, 1990],
     [-406, 1858],
     [-410, 1857]]},
   'properties': {'road_type': 'Secondary road',
    'traffic_level': 0.628000020980835,
    'traffic_road_coverage': 'full'},
   'id': 0,
   'type': 2},
  {'geometry': {'type': 'LineString',
    'coordinates': [[2260, 2004],
     [2292, 1672],
     [2304, 1600],
     [2316, 1550],
     [2346, 1456],
     [2382, 1362],
     [2504

In [6]:
def extract_JSONvalues(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

 

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

 

    results = extract(obj, arr, key)
    return results

In [8]:
np.average(extract_JSONvalues(trfficData, 'traffic_level'))

0.7102963024819339

## Ricky Brundritt from Microsoft about Whole City Traffic Download:


Is it possible to have the request in such a form that I get the traffic data for a city? If possible, would you please help me understand it how for example for Berlin I should setup the HTTP request? I am assuming this might be possible by rough estimation of the tile (zoom, x, y) in the HTTP request? Or maybe easier? You will need to download all the tiles that cover a full city. This is the only way to directly access large amounts of traffic data from Azure Maps. If you look at the tile grid doc, I recommend using the get quadkeys in bounding box function and then convert these to x, y, z values.

# Quadkey-indices - up next...

- https://docs.microsoft.com/en-us/azure/azure-maps/zoom-levels-and-tile-grid?tabs=csharp#quadkey-indices